# Lab 3 — clustering

In [ ]:
# Given Imports
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle as pk

import random as rn
# Import bokeh
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()
# Categorial colors
from bokeh.palettes import Dark2_8

%matplotlib inline
plt.style.use("ggplot")

### Exercise 3.11 : Clustering tags

In [ ]:
# Let's open the pickle to load it.
with open( "tagCoord.pickle", "rb" ) as f:
    tagEmbed = pk.load( f, encoding="utf-8" )

In [ ]:
#Cluster the data using the k-means clustering algorithm.

# Input: N data points {x1, ..., xn} , given in our pickle.
#        K number of clusters, between 2 and 5 for us.
# Output: We will have mu_k, center of clusters, K times.
#         r_nk, point-cluster assigment indicator. If r_nk = 1, n is in cluster k.

# r_nk = 1 only for k = argmin ||xn - mu_k||
# mu_k = sum_over_n(r_nk + xn)/sum_over_n(r_nk)

# Start with K = 2
def k_mean(k, data, give_up = 1000):
    mat = np.array(list(zip(*data))[1])
    
    #We arbitrary decide that if the centroids all move less than 1e-5 * the variance, they have converged and we can stop.
    var = np.var(mat,1)
    thershold = np.linalg.norm(var)*1e-5
    
    #Pick k random data points as initialposition for centroids.
    centroids_idx = np.random.randint(mat.shape[0], size=k)
    centroids = mat[centroids_idx]
    
    ditances = np.zeros((mat.shape[0],k))
    
    # We should loop until convergence but just in case we do not , we set a maximal number of step.
    for step in range(0,give_up):
        
        #For each centroid, the distance between each point and this centroids.
        for i in range(0, k):
            ditances[:,i] = np.linalg.norm(mat - centroids[i,:],axis=1)
        
        #The index of the closet centroids for each point
        closest = np.argpartition(ditances,1,1)[:,0]
        
        #For each centroid, the center of gravity of all point closest to this centroid. 
        means = np.zeros(centroids.shape)
        for i in range(0, k):
            means[i,:] = np.mean(mat[closest == i,:],0)
            
        #We will set the new centroids as the center of gravity of the cluster we found.
        #But first we compute the distance between the current position of each centroid and its position for the next step.
        move = means - centroids
        dist = np.max(np.linalg.norm(move,axis=0))
        
        #If no centroid have moved more than the thershold, we are close enough from the convergence point.
        if(dist < thershold):
            return closest
        centroids = means
    print("Centroid have not stabilized after",give_up,"steps")
    return closest
data = list(tagEmbed.items())
clustering = k_mean(3,data)

    

In [ ]:
# Use the categorial colors for clustering.
main_dims = (0,1)
print(len(clustering))
source = ColumnDataSource(
    data={
        "x": [x[1][main_dims[0]] for x in data],
        "y": [x[1][main_dims[1]] for x in data],
        "name": [x[0] for x in data],
        "color": [Dark2_8[x] for x in clustering],
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]
print("minY",minY)
p = figure(plot_width=960, plot_height=360, tools=tools, title="Mouse over the dots")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p, notebook_handle=True)

### Exercise 3.12 : Clustering movies

In [ ]:
# k-medioids algorithm implementation
# X, set of points, k number of clusters. We don't give d as we will use Jaccard.
def kmedAlgo(X, k):
    # Initialize medioids
    m = list()
    for i in range(1, k):
        m[i] = random.sample(X, 1)
    C = list()
    while True:
        for i in range(1, k):
            C[i] = list()
        # Assign points to clusters
        for x in X:
            #TODO j = qqch
            C[j] = C[j].append(x)
        # Recompute medioids
        for i in range(1, k):
            #TODO m[i] = qqch
        if convergence:
            break